<a href="https://colab.research.google.com/github/vit10load/machine-learning-python/blob/master/C%C3%B3pia_de_Testing_classifier_symmetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from keras import *
from keras.models import *
from keras.layers import Input, Dense
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
import numpy as np
from keras.utils.np_utils import to_categorical
import glob
import os.path
from keras.models import load_model
import cv2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout, Flatten, Input, Dense
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import Nadam
from google.colab import drive
from IPython.display import display
from PIL import Image


drive.mount('/gdrive', force_remount=True);


Mounted at /gdrive


In [50]:
#inicialização de uma rede sequencial
classifier = InceptionResNetV2()

In [51]:

classifier.summary();



Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_409 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 149, 149, 32) 96          conv2d_409[0][0]                 
__________________________________________________________________________________________________
activation_411 (Activation)     (None, 149, 149, 32) 0           batch_normalization_406[0][0]    
________________________________________________________________________________

In [52]:
#training_set = train_datagen.flow_from_directory(path_images_dont_simetric,
                                                #target_size=(64,64),
                                                #batch_size= 32,
                                                #class_mode=None)
'''
test_set = test_datagen.flow_from_directory(path_images_simetric,
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode =None)
'''

naosimetric_dir = glob.glob(os.path.join('/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA', '*'))

print(naosimetric_dir)
label_naosimetric = np.zeros(len(naosimetric_dir))

# indicar as imagens de treinamento para simetria e associar à classe 1 (um)
simetric_dir = glob.glob(os.path.join('/gdrive/My Drive/PROJECTS/test/SIMETRICA', '*'))
label_simetric = np.ones(len(simetric_dir))

# lendo as imagens das pastas arara e capivara do meu dataset
X_path = naosimetric_dir + simetric_dir
X = []
for f in X_path:
    X.append(np.array(cv2.resize(cv2.imread(f), (299,299), interpolation = cv2.INTER_AREA))) 
    
X = np.array(X)
X = X / 255 # normalizar a imagem

y = np.concatenate((label_naosimetric, label_simetric))
y = to_categorical(y, 2)

classes_existentes = ['NAOSIMETRICA', 'SIMETRICA']




['/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD035_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD076_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD078_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD075_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD033_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD157_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD210_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD006_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD339_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD368_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD312_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD304_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD388_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD254_lesion.bmp', '/gdrive/My Drive/PROJECTS/train/NAOSIMETRICA/IMD370_lesion.bmp', '/gdrive/

In [53]:
# dividir para treinamento e teste: 80% para treinamento e 20% para teste
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42)

# aumentar dados de treinamento para evitar overfitting
datagen = ImageDataGenerator(
        zoom_range = 0.1, # Zoom aleatório
        rotation_range= 15, 
        width_shift_range=0.1,  # shift horizontal
        height_shift_range=0.1,  # shift vertical
        horizontal_flip=True,  
        vertical_flip=True)

datagen.fit(X_train)

# congelando as camadas pre-treinadas do modelo para manter os pesos que le já aprendeu no treinamento original
x = classifier.output
x = Flatten()(x)
x = Dense(3078,activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(256,activation='relu')(x) 
x = Dropout(0.2)(x)
out = Dense(2,activation='softmax')(x)

modelo_com_transferencia=Model(inputs=classifier.input,outputs=out)

In [54]:
modelo_com_transferencia.compile(optimizer = Nadam(lr = 0.0001) , loss = 'categorical_crossentropy', metrics=['accuracy'])
history = modelo_com_transferencia.fit(X_train, y_train, batch_size = 1, epochs = 100, initial_epoch = 0, validation_data = (X_val, y_val))
print(history)


Epoch 1/2
1/1 [==============================] - 93s 93s/step - loss: 0.6933 - accuracy: 0.4444
Epoch 2/2
1/1 [==============================] - 35s 35s/step - loss: 0.6913 - accuracy: 0.7778


In [55]:
modelo_com_transferencia.save('model');

INFO:tensorflow:Assets written to: model/assets


In [56]:

modelo_com_transferencia = load_model('model');



In [64]:
#carrego a imagem de teste
path_test = "/gdrive/My Drive/PROJECTS/IMD003_lesion.bmp"

test_image = image.load_img(path_test,target_size =(299,299));
#convertendo o objeto da imagem para um array
test_image = image.img_to_array(test_image);
# expandir dimensões do array, ajuda a prever os resultados pelo modelo
test_image = test_image.reshape((test_image.shape[0], test_image.shape[1], test_image.shape[2]))/255
test_image = np.expand_dims(test_image, axis = 0);
result = modelo_com_transferencia.predict(test_image);
result= np.argmax(result, axis = 1)

print(result);

print(classes_existentes[result[0]])

# 0 >= NAOSIMETRICA
# 1 >= SIMETRICA




[0]
NAOSIMETRICA
